In [1]:
import torch
import torch.nn as nn
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 2)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
model = SimpleModel()
model.eval()

SimpleModel(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)

In [2]:
from torch.export import export
dummy_input = torch.randn(2, 10)
exported_program = export(model, args=(dummy_input,))

In [3]:
print(exported_program)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_fc1_weight: "f32[5, 10]", p_fc1_bias: "f32[5]", p_fc2_weight: "f32[2, 5]", p_fc2_bias: "f32[2]", x: "f32[2, 10]"):
             # File: /var/folders/n1/ygzk0n895x15grwb5rm1ld300000gn/T/ipykernel_38527/3691556958.py:10 in forward, code: x = self.fc1(x)
            linear: "f32[2, 5]" = torch.ops.aten.linear.default(x, p_fc1_weight, p_fc1_bias);  x = p_fc1_weight = p_fc1_bias = None
            
             # File: /var/folders/n1/ygzk0n895x15grwb5rm1ld300000gn/T/ipykernel_38527/3691556958.py:11 in forward, code: x = self.relu(x)
            relu: "f32[2, 5]" = torch.ops.aten.relu.default(linear);  linear = None
            
             # File: /var/folders/n1/ygzk0n895x15grwb5rm1ld300000gn/T/ipykernel_38527/3691556958.py:12 in forward, code: x = self.fc2(x)
            linear_1: "f32[2, 2]" = torch.ops.aten.linear.default(relu, p_fc2_weight, p_fc2_bias);  relu = p_fc2_weight = p_fc2_bias = None
      

In [4]:
exported_program.module()(dummy_input)

tensor([[0.0831, 0.1144],
        [0.1207, 0.4827]], grad_fn=<AddmmBackward0>)

In [5]:
import os
output_path = torch._inductor.aoti_compile_and_package(
    exported_program,
    package_path=os.path.join(os.getcwd(), "model.pt2"),
)

In [6]:
model = torch._inductor.aoti_load_package(os.path.join(os.getcwd(), "model.pt2"))
model(dummy_input)

tensor([[0.0831, 0.1144],
        [0.1207, 0.4827]])